### RUL Target Computation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
# 1. Load rolling windows sequence (NumPy array)
sequences = np.load('/content/drive/MyDrive/PrognosAI_OCT25/Data/rolling_window_sequences.npy')
print(f"Loaded rolling window sequences with shape: {sequences.shape}")

Loaded rolling window sequences with shape: (152559, 30, 66)


In [4]:
# 2. Load sequence metadata (engine_id, cycle)
metadata_df = pd.read_csv('/content/drive/MyDrive/PrognosAI_OCT25/Data/sequence_metadata.csv')
print(f"Loaded sequence metadata with shape: {metadata_df.shape}")
metadata_df.head()

Loaded sequence metadata with shape: (152559, 2)


,engine_id,cycle
0,1,8.0
1,1,8.0
2,1,9.0
3,1,9.0
4,1,9.0


In [5]:
# 3. Calculate Remaining Useful Life (RUL) per sequence

# Find max cycle per engine in metadata
max_cycle_per_engine = metadata_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['engine_id', 'max_cycle']
max_cycle_per_engine.head()

,engine_id,max_cycle
0,1,321.0
1,2,299.0
2,3,307.0
3,4,274.0
4,5,269.0


In [6]:
# Merge max cycle info with metadata
metadata_df = metadata_df.merge(max_cycle_per_engine, on='engine_id', how='left')
metadata_df.head()


,engine_id,cycle,max_cycle
0,1,8.0,321.0
1,1,8.0,321.0
2,1,9.0,321.0
3,1,9.0,321.0
4,1,9.0,321.0


In [7]:
# Compute RUL
metadata_df['RUL'] = metadata_df['max_cycle'] - metadata_df['cycle']

In [8]:
metadata_df.head()

,engine_id,cycle,max_cycle,RUL
0,1,8.0,321.0,313.0
1,1,8.0,321.0,313.0
2,1,9.0,321.0,312.0
3,1,9.0,321.0,312.0
4,1,9.0,321.0,312.0


In [9]:
# Optional: Cap RUL values
# rul_cap = 130
# metadata_df['RUL'] = metadata_df['RUL'].clip(upper=rul_cap)

# Drop max_cycle column if not needed further
metadata_df.drop(columns=['max_cycle'], inplace=True)

metadata_df.head()

,engine_id,cycle,RUL
0,1,8.0,313.0
1,1,8.0,313.0
2,1,9.0,312.0
3,1,9.0,312.0
4,1,9.0,312.0


In [10]:
# 4. Validate by inspecting example engine(s)
engines_to_check = metadata_df['engine_id'].unique()[:3]  # first 3 engines
for engine in engines_to_check:
    sample = metadata_df[metadata_df['engine_id'] == engine][['cycle', 'RUL']]
    print(f"\nEngine {engine} - Cycles and RULs:")
    print(sample.head(10))
    print(sample.tail(10))


Engine 1 - Cycles and RULs:
   cycle    RUL
0    8.0  313.0
1    8.0  313.0
2    9.0  312.0
3    9.0  312.0
4    9.0  312.0
5    9.0  312.0
6   10.0  311.0
7   10.0  311.0
8   10.0  311.0
9   10.0  311.0
     cycle  RUL
881  312.0  9.0
882  313.0  8.0
883  314.0  7.0
884  315.0  6.0
885  316.0  5.0
886  317.0  4.0
887  318.0  3.0
888  319.0  2.0
889  320.0  1.0
890  321.0  0.0

Engine 2 - Cycles and RULs:
     cycle    RUL
891    8.0  291.0
892    8.0  291.0
893    9.0  290.0
894    9.0  290.0
895    9.0  290.0
896    9.0  290.0
897   10.0  289.0
898   10.0  289.0
899   10.0  289.0
900   10.0  289.0
      cycle  RUL
1959  290.0  9.0
1960  291.0  8.0
1961  292.0  7.0
1962  293.0  6.0
1963  294.0  5.0
1964  295.0  4.0
1965  296.0  3.0
1966  297.0  2.0
1967  298.0  1.0
1968  299.0  0.0

Engine 3 - Cycles and RULs:
      cycle    RUL
1969    8.0  299.0
1970    8.0  299.0
1971    9.0  298.0
1972    9.0  298.0
1973    9.0  298.0
1974    9.0  298.0
1975   10.0  297.0
1976   10.0  297.0
1977 

In [11]:
# 5. Save updated metadata with RUL target
metadata_df.to_csv('/content/drive/MyDrive/PrognosAI_OCT25/Data/sequence_metadata_with_RUL.csv', index=False)
print("Saved updated metadata with RUL as 'sequence_metadata_with_RUL.csv'")

Saved updated metadata with RUL as 'sequence_metadata_with_RUL.csv'


**Process Steps:**  
- Loaded rolling window feature sequences from 'rolling_windows_feature.npy'.  
- Loaded sequence metadata from 'sequence_metadata.csv' including engine and cycle identifiers.  
- Computed Remaining Useful Life (RUL) for each sequence as the difference between
  maximum cycle for the engine and the current cycle.  
- Optionally applied RUL capping at 130 to stabilize regression targets.  
- Saved updated metadata including RUL targets for downstream model training.

**Outputs:**  
- Rolling window sequences (NumPy array) ready for modeling.  
- Updated sequence metadata with assigned RUL targets.  

**Next Steps:**  
- Use these datasets for training and evaluating LSTM/GRU predictive models for RUL.  

---